In [5]:
import copy

variables = ['WA', 'NT', 'SA', 'QLD', 'NSW', 'VIC', 'TAS']
colors = ['R', 'G', 'B']

adjacency = {
    'WA': ['NT', 'SA'],
    'NT': ['WA', 'SA', 'QLD'],
    'SA': ['WA', 'NT', 'QLD', 'NSW', 'VIC'],
    'QLD': ['NT', 'SA', 'NSW'],
    'NSW': ['SA', 'QLD', 'VIC'],
    'VIC': ['SA', 'NSW'],
    'TAS': []
}

def init_domains():
    return {var: list(colors) for var in variables}

def is_valid(var, color, assignment):
    return all(assignment.get(neigh) != color for neigh in adjacency[var])

def select_unassigned_variable(assignment, domains):
    unassigned = [v for v in variables if v not in assignment]
    # MRV
    mrv_vars = sorted(unassigned, key=lambda var: len(domains[var]))
    min_len = len(domains[mrv_vars[0]])
    mrv_candidates = [v for v in mrv_vars if len(domains[v]) == min_len]
    # Degree heuristic
    return max(mrv_candidates, key=lambda var: len([n for n in adjacency[var] if n not in assignment]))

def forward_check(var, color, domains, assignment):
    removed = {}
    for neighbor in adjacency[var]:
        if neighbor not in assignment and color in domains[neighbor]:
            domains[neighbor].remove(color)
            removed.setdefault(neighbor, []).append(color)
            if not domains[neighbor]:  # domain wipeout
                return False, removed
    return True, removed

def restore_domains(domains, removed):
    for var, values in removed.items():
        domains[var].extend(values)

def backtrack(assignment, domains):
    if len(assignment) == len(variables):
        return assignment

    var = select_unassigned_variable(assignment, domains)
    for color in domains[var]:
        if is_valid(var, color, assignment):
            assignment[var] = color
            new_domains = copy.deepcopy(domains)  # deepcopy to isolate
            success, removed = forward_check(var, color, new_domains, assignment)
            if success:
                result = backtrack(assignment, new_domains)
                if result:
                    return result
            del assignment[var]
    return None

def main():
    domains = init_domains()
    solution = backtrack({}, domains)
    if solution:
        print("Valid Solution Found:")
        for state in variables:
            print(f"{state}: {solution[state]}")
        print("No two adjacent regions share the same color.")
    else:
        print(" No valid solution found.")

if __name__ == "__main__":
    main()



Valid Solution Found:
WA: B
NT: G
SA: R
QLD: B
NSW: G
VIC: B
TAS: R
No two adjacent regions share the same color.
